In [34]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc, rcParams
import seaborn as sns
import glob
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
# import lightgbm
from lightgbm import LGBMRegressor
import datetime

import warnings
warnings.filterwarnings('ignore')

plt.style.use('ggplot')
pd.set_option('max_columns', 100)
pd.set_option("display.precision", 4)
rcParams['figure.figsize'] = (16, 8)
rc('font', family='AppleGothic')

In [35]:
train_path = './trainingdata'
infer_path = './inferencedata'
submission_path = './submission'
importance_path = './feature_importance'
oof_path = './oof_preds'
pil_path = './magok'

In [36]:
submission = pd.read_csv('./data/sample_submission.csv')
submission2 = pd.read_csv('./data/sample_submission.csv')

In [37]:
# 테스트 time_str
time_str = '20210105_221155_변수소거테스트'

In [72]:
time_str = '20210112_220351'
train = pd.read_csv(
    os.path.join(train_path, f'training_{time_str}.csv')
)
X_test = pd.read_csv(
    os.path.join(infer_path, f'inference_{time_str}.csv')
)

In [73]:
train_day23 = pd.read_csv(
    os.path.join(train_path, f'training_{time_str}_day23.csv')
)
X_test_day23 = pd.read_csv(
    os.path.join(infer_path, f'inference_{time_str}_day23.csv')
)

In [74]:
train['Hour'] = train['Hour'].astype('category')
train_day23['Hour'] = train_day23['Hour'].astype('category')
X_test['Hour'] = X_test['Hour'].astype('category')
X_test_day23['Hour'] = X_test_day23['Hour'].astype('category')

# Metric

In [59]:
def pinball(pred, actual):
    quantile_col = pred.columns.tolist()
    pred2 = pred.copy()
    actual2 = actual.copy()
    
    pred2.index = range(len(pred2))
    actual2.index = range(len(actual2))

    pinball_loss = []
    for item in quantile_col:
        pre_pinball = []
        for idx in range(len(pred2)):
            y = actual2.iloc[idx]
            z = pred2[item].iloc[idx]
            q = item
            
            if y >= z:
                value = (y - z)*q
                pre_pinball.append(value)
            else:
                value = (z - y)*(1 - q)
                pre_pinball.append(value)
        loss_mean = np.mean(pre_pinball)
        pinball_loss.append(loss_mean)
    
    fin = np.mean(pinball_loss)
    print(f'Pinball Loss: {fin}')
    
    return pinball_loss         

# Inference

In [60]:
# X_train = X_train.drop(columns = drop_col)
# X_test = X_test.drop(columns = drop_col)

In [77]:
X_train = train.drop(columns = ['1day_after_target', '2day_after_target'])
target_1 = train['1day_after_target']
target_2 = train['2day_after_target']

In [78]:
X_train_day23 = train_day23.drop(columns = ['1day_after_target', '2day_after_target'])
target_1_day23 = train_day23['1day_after_target']
target_2_day23 = train_day23['2day_after_target']

In [63]:
def LGBM_reg(X_train,y_train, test, target_num):
    quantiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    
    # 랜덤모수 샘플 추론이 왜 안되지;;;
    # fold = StratifiedKFold(n_splits=5, shuffle = True)
    
    fold = KFold(n_splits=7, shuffle = True)

    oof_preds = np.zeros([X_train.shape[0], len(quantiles)])
    
    feature_importance_df = pd.DataFrame(np.zeros([len(X_train.columns), len(quantiles)]))
    feature_importance_df.columns = quantiles
    
    test_df = pd.DataFrame(np.zeros([test.shape[0], len(quantiles)]))
    test_df.columns = quantiles
    
    fold_metric = []
    # training
    target = y_train
    for fold_, (train_idx, valid_idx) in enumerate(
        fold.split(np.array(X_train), target)
    ):
        
#         if fold_ >0:
#             break
        train_x, train_y = X_train.iloc[train_idx], target.iloc[train_idx]
        valid_x, valid_y = X_train.iloc[valid_idx], target.iloc[valid_idx]

        oof_df = pd.DataFrame()
        for idx, q in enumerate(quantiles):
            print(f'\nquantile: {q}\n')
            model = LGBMRegressor(objective='quantile',
                                  alpha=q,
                                  n_estimators=10000, 
                                  bagging_fraction=0.7, 
                                  learning_rate=0.027, 
                                  subsample=0.7
                                 )   

            model.fit(train_x, train_y, eval_metric = ['quantile'], 
              eval_set=[(valid_x, valid_y)], early_stopping_rounds=300, verbose=1500)

            oof_preds[[valid_idx],idx] = \
                model.predict(valid_x).round(2)

            oof_df[q] = model.predict(valid_x).round(2)
            test_df[q] += model.predict(test).round(2) / fold.n_splits
            
            feature_importance_df[q] = model.feature_importances_ / 7
            
        metric = pinball(oof_df, valid_y)
        loss = np.mean(metric)
        print(f'\n FOLD {fold_}의 total pinball loss: {loss}\n')
        fold_metric.append(metric)
        # feature importance
        
        print('\n===================================================')
        print(f'FOLD {fold_} Success')
        print('===================================================\n')
        
    feature_importance_df.index = X_train.columns.tolist()
    feature_importance_df2 = feature_importance_df.reset_index()
    
    feature_importance_df2.to_csv(f'{importance_path}/{time_str}_{target_num}_fe.csv', index = False)
    pd.DataFrame(oof_preds).to_csv(f'{oof_path}/{time_str}_{target_num}_oof_pred.csv', index = False)
    
    return oof_preds, fold_metric, test_df, feature_importance_df2

In [79]:
oof_preds, fold_metric, test_df, feature_imp = \
    LGBM_reg(X_train, target_1, X_test,1)


quantile: 0.1

Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[372]	valid_0's quantile: 1.34505

quantile: 0.2

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.061
[3000]	valid_0's quantile: 2.05163
Early stopping, best iteration is:
[3294]	valid_0's quantile: 2.05005

quantile: 0.3

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.42452
Early stopping, best iteration is:
[2435]	valid_0's quantile: 2.39891

quantile: 0.4

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.51214
[3000]	valid_0's quantile: 2.48928
[4500]	valid_0's quantile: 2.47951
[6000]	valid_0's quantile: 2.47341
Early stopping, best iteration is:
[5724]	valid_0's quantile: 2.47292

quantile: 0.5

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.40945
Early stopping, best iteration is:
[1516]	valid_0's q

Early stopping, best iteration is:
[435]	valid_0's quantile: 1.37972

quantile: 0.2

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.10562
[3000]	valid_0's quantile: 2.09623
Early stopping, best iteration is:
[3192]	valid_0's quantile: 2.09269

quantile: 0.3

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.44672
[3000]	valid_0's quantile: 2.42174
Early stopping, best iteration is:
[2776]	valid_0's quantile: 2.42075

quantile: 0.4

Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[865]	valid_0's quantile: 2.58339

quantile: 0.5

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.45785
[3000]	valid_0's quantile: 2.43368
Early stopping, best iteration is:
[3735]	valid_0's quantile: 2.42741

quantile: 0.6

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.19311
[3000]	valid_0's 

In [80]:
oof_preds2, fold_metric2, test_df2, feature_imp2 = \
    LGBM_reg(X_train, target_2, X_test,2)


quantile: 0.1

Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[551]	valid_0's quantile: 1.35747

quantile: 0.2

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.11338
[3000]	valid_0's quantile: 2.0948
Early stopping, best iteration is:
[3113]	valid_0's quantile: 2.09371

quantile: 0.3

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.48613
Early stopping, best iteration is:
[2066]	valid_0's quantile: 2.45813

quantile: 0.4

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.6188
[3000]	valid_0's quantile: 2.58176
[4500]	valid_0's quantile: 2.5629
[6000]	valid_0's quantile: 2.55067
[7500]	valid_0's quantile: 2.54275
[9000]	valid_0's quantile: 2.53319
Did not meet early stopping. Best iteration is:
[10000]	valid_0's quantile: 2.53198

quantile: 0.5

Training until validation scores don't improve for 300 rounds
[1500

[1500]	valid_0's quantile: 2.3371
[3000]	valid_0's quantile: 2.3156
Early stopping, best iteration is:
[3953]	valid_0's quantile: 2.30783

quantile: 0.7

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 1.91414
[3000]	valid_0's quantile: 1.90895
Early stopping, best iteration is:
[2753]	valid_0's quantile: 1.90632

quantile: 0.8

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 1.36725
Early stopping, best iteration is:
[2630]	valid_0's quantile: 1.36351

quantile: 0.9

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 0.73149
Early stopping, best iteration is:
[1527]	valid_0's quantile: 0.731421
Pinball Loss: 1.9505679664042535

 FOLD 3의 total pinball loss: 1.9505679664042535


FOLD 3 Success


quantile: 0.1

Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[508]	valid_0's quantile: 1.34905

quantile: 0.2

Training until

In [81]:
oof_preds_day23, fold_metric_day23, test_df_day23, feature_imp_day23 = \
    LGBM_reg(X_train_day23, target_1_day23, X_test_day23,11)


quantile: 0.1

Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[700]	valid_0's quantile: 1.35991

quantile: 0.2

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.10064
Early stopping, best iteration is:
[1332]	valid_0's quantile: 2.09929

quantile: 0.3

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.47239
[3000]	valid_0's quantile: 2.4415
[4500]	valid_0's quantile: 2.42098
Early stopping, best iteration is:
[5634]	valid_0's quantile: 2.41746

quantile: 0.4

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.58064
[3000]	valid_0's quantile: 2.54015
[4500]	valid_0's quantile: 2.52167
Early stopping, best iteration is:
[4745]	valid_0's quantile: 2.52013

quantile: 0.5

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.47471
[3000]	valid_0's quantile: 2.43844
[4500]	valid_0's 

[1500]	valid_0's quantile: 1.83724
Early stopping, best iteration is:
[2670]	valid_0's quantile: 1.82625

quantile: 0.8

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 1.3091
Early stopping, best iteration is:
[1989]	valid_0's quantile: 1.30629

quantile: 0.9

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 0.708424
Early stopping, best iteration is:
[2436]	valid_0's quantile: 0.707541
Pinball Loss: 1.9033874623078748

 FOLD 3의 total pinball loss: 1.9033874623078748


FOLD 3 Success


quantile: 0.1

Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[653]	valid_0's quantile: 1.37528

quantile: 0.2

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.13415
Early stopping, best iteration is:
[1773]	valid_0's quantile: 2.12722

quantile: 0.3

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's qua

In [82]:
oof_preds2_day23, fold_metric2_day23, test_df2_day23, feature_imp2_day23 = \
    LGBM_reg(X_train_day23, target_2_day23, X_test_day23,22)


quantile: 0.1

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 1.36707
Early stopping, best iteration is:
[1380]	valid_0's quantile: 1.36623

quantile: 0.2

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.15057
Early stopping, best iteration is:
[1414]	valid_0's quantile: 2.15041

quantile: 0.3

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.49736
[3000]	valid_0's quantile: 2.47805
[4500]	valid_0's quantile: 2.46828
[6000]	valid_0's quantile: 2.46447
Early stopping, best iteration is:
[5785]	valid_0's quantile: 2.46412

quantile: 0.4

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.60705
[3000]	valid_0's quantile: 2.58232
[4500]	valid_0's quantile: 2.55896
[6000]	valid_0's quantile: 2.54517
Early stopping, best iteration is:
[7068]	valid_0's quantile: 2.53903

quantile: 0.5

Training until validation scores don't imp

[1500]	valid_0's quantile: 1.82921
[3000]	valid_0's quantile: 1.81194
Early stopping, best iteration is:
[3373]	valid_0's quantile: 1.80926

quantile: 0.8

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 1.32089
[3000]	valid_0's quantile: 1.30792
Early stopping, best iteration is:
[4103]	valid_0's quantile: 1.30189

quantile: 0.9

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 0.723854
[3000]	valid_0's quantile: 0.721559
Early stopping, best iteration is:
[3474]	valid_0's quantile: 0.721208
Pinball Loss: 1.8626755026004074

 FOLD 3의 total pinball loss: 1.8626755026004074


FOLD 3 Success


quantile: 0.1

Training until validation scores don't improve for 300 rounds
Early stopping, best iteration is:
[650]	valid_0's quantile: 1.3447

quantile: 0.2

Training until validation scores don't improve for 300 rounds
[1500]	valid_0's quantile: 2.07582
[3000]	valid_0's quantile: 2.06023
Early stopping, best iterat

# submission

In [83]:
submission.loc[submission.id.str.contains("Day7"), "q_0.1":] = test_df.sort_index().values
submission.loc[submission.id.str.contains("Day8"), "q_0.1":] = test_df2.sort_index().values
submission

,id,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
0,0.csv_Day7_0h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.csv_Day7_0h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.csv_Day7_1h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.csv_Day7_1h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.csv_Day7_2h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
7771,80.csv_Day8_21h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7772,80.csv_Day8_22h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7773,80.csv_Day8_22h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7774,80.csv_Day8_23h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [84]:
submission2.loc[submission2.id.str.contains("Day7"), "q_0.1":] = test_df_day23.sort_index().values
submission2.loc[submission2.id.str.contains("Day8"), "q_0.1":] = test_df2_day23.sort_index().values
submission2

,id,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
0,0.csv_Day7_0h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.csv_Day7_0h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.csv_Day7_1h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.csv_Day7_1h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.csv_Day7_2h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
7771,80.csv_Day8_21h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7772,80.csv_Day8_22h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7773,80.csv_Day8_22h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7774,80.csv_Day8_23h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Ensemble

In [85]:
submission3 = submission[['id']]
sub_value = submission.iloc[:,1:] * 0.7 + submission2.iloc[:,1:] * 0.3

submission3 = pd.concat([submission3,sub_value],axis = 1)

# 제출

In [63]:
# time_str = datetime.datetime.strftime(
#     datetime.datetime.today(),
#     '%Y%m%d_%H%M%S'
# )

# training 데이터 저장한 시간이랑 맞춰서 사용
submission.to_csv(
    os.path.join(submission_path, f'제출용_{time_str}.csv'), 
    index=False
)

In [87]:
# 앙상블 결과 제출
submission3.to_csv(
    os.path.join(submission_path, f'제출용_{time_str}_ensemble_가중평균.csv'), 
    index=False
)

- 가중평균: 0.7/0.3
- 가중평균2: 0.8/0.2
- 가중평균3: 0.6/0.4

# 외부 데이터와 앙상블(magok)

In [91]:
# 필령이형 파일 앙상블
time_str = '20210110_225651'
time_str2 = '20210112_220351'
temp = pd.read_csv(os.path.join(submission_path, f'제출용_{time_str}_ensemble_가중평균.csv'))
target = pd.read_csv(os.path.join(submission_path, f'제출용_{time_str2}_ensemble_가중평균.csv'))

target.iloc[:,1:] = temp.iloc[:,1:] * 0.3 + target.iloc[:,1:] * 0.7

In [92]:
target.to_csv(
    os.path.join(submission_path, f'제출용_{time_str2}_ensemble_external_submission_ensemble_v2.csv'), 
    index=False
)

# 변수중요도 데이터 추출

In [46]:
time_str = '20210105_221155'
imp_path1 = f'{importance_path}/{time_str}_1_fe.csv'
imp_path2 = f'{importance_path}/{time_str}_2_fe.csv'

imp1 = pd.read_csv(
    imp_path1
)
imp2 = pd.read_csv(
    imp_path2
)

In [48]:
drop_col = ['DNI_label','TARGET_interval','2days_mean_TARGET_rolling']

# inference data load

In [216]:
infer_time = '20210107_231340'
temp = pd.read_csv(
    os.path.join(submission_path,f'제출용_{infer_time}_ensemble_가중평균.csv')
)    

In [219]:
submission3[submission3['q_0.2'] > 0]

,id,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
1464,15.csv_Day7_12h00m,0.0583,0.0317,0.4843,0.2177,0.1127,-0.3424,-0.5417,-0.1243,0.3677
1465,15.csv_Day7_12h30m,0.0000,0.1504,0.3266,0.1161,0.1414,-0.3514,-0.3390,0.0201,0.3806
1483,15.csv_Day7_21h30m,0.0000,0.0947,0.3749,-0.1961,-0.6714,-0.7457,-0.7140,0.0957,0.6906
1487,15.csv_Day7_23h30m,0.5910,0.1130,-0.8010,-0.2110,-0.9950,-1.4290,-1.4220,1.2080,1.4630
1512,15.csv_Day8_12h00m,0.6081,0.9960,0.5841,0.4749,0.3951,0.3527,0.1316,0.3450,0.3909
...,...,...,...,...,...,...,...,...,...,...
6455,67.csv_Day7_11h30m,0.0000,0.0440,0.3720,-0.2900,-0.3560,-0.4870,-0.9970,0.2280,0.5550
6480,67.csv_Day8_0h00m,0.1240,0.1250,0.1100,0.1360,0.2830,0.3890,0.3810,0.2710,0.4630
6481,67.csv_Day8_0h30m,-0.0220,0.0430,-0.2070,-0.3850,-1.1430,-1.4040,-0.8530,0.1530,0.1000
6502,67.csv_Day8_11h00m,0.6560,0.2460,0.8870,1.0260,0.6070,-1.0190,0.5090,0.9180,1.2640


In [230]:
temp

,id,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
0,0.csv_Day7_0h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.csv_Day7_0h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.csv_Day7_1h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.csv_Day7_1h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.csv_Day7_2h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
7771,80.csv_Day8_21h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7772,80.csv_Day8_22h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7773,80.csv_Day8_22h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7774,80.csv_Day8_23h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
